In [9]:
from ecologico import Ecologico
from agente import Agente
from estrategias import *

In [10]:
def create_random_adjacency_list(n, k):
    edges = set()
    
    while len(edges) < k:
        u = random.randint(0, n-1)
        v = random.randint(0, n-1)
        if u != v:
            edge = tuple(sorted((u, v)))  # Sort the tuple to avoid (u, v) and (v, u) as different
            edges.add(edge)
    
    return list(edges)

create_random_adjacency_list(25, 75)

[(6, 12),
 (6, 15),
 (7, 20),
 (4, 9),
 (12, 16),
 (3, 7),
 (4, 12),
 (21, 22),
 (5, 7),
 (22, 23),
 (3, 10),
 (8, 9),
 (8, 12),
 (11, 23),
 (8, 24),
 (1, 18),
 (9, 23),
 (0, 20),
 (2, 23),
 (13, 20),
 (7, 16),
 (7, 13),
 (0, 18),
 (7, 22),
 (5, 6),
 (4, 8),
 (12, 15),
 (3, 12),
 (3, 18),
 (12, 24),
 (3, 21),
 (5, 12),
 (0, 1),
 (14, 24),
 (5, 21),
 (1, 2),
 (0, 16),
 (10, 14),
 (22, 24),
 (5, 24),
 (1, 14),
 (1, 20),
 (11, 22),
 (2, 13),
 (15, 19),
 (16, 18),
 (7, 18),
 (7, 24),
 (12, 14),
 (20, 21),
 (3, 14),
 (5, 11),
 (12, 20),
 (3, 17),
 (14, 23),
 (3, 23),
 (5, 8),
 (14, 20),
 (5, 17),
 (8, 16),
 (8, 10),
 (9, 15),
 (2, 3),
 (17, 22),
 (8, 19),
 (1, 13),
 (9, 24),
 (2, 12),
 (11, 15),
 (13, 18),
 (11, 18),
 (8, 22),
 (2, 21),
 (9, 18),
 (1, 22)]

In [11]:
estrategias = {
    "azarosa" : (estrat_proponer_rand, estrat_aceptar_rand, 5),
    "mala_onda" : (estrat_proponer_rand, estrat_aceptar_nunca, 5),
    "promedio" : (estrat_proponer_promedio, estrat_aceptar_promedio, 5),
    "rata" : (estrat_rata, estrat_aceptar_promedio, 5),
    "antirrata" : (estrat_antirrata_proponer, estrat_antirrata_aceptar, 5)
    }   



ecologico = Ecologico(50, 10, estrategias, 25, create_random_adjacency_list(25, 75))


In [12]:
ecologico.competir(ecologico.estrategias)

Cantidad de estrategias azarosa: 5
Cantidad de estrategias mala_onda: 5
Cantidad de estrategias promedio: 5
Cantidad de estrategias rata: 5
Cantidad de estrategias antirrata: 5
Cantidad de estrategias azarosa: 5.0
Cantidad de estrategias mala_onda: 1.0
Cantidad de estrategias promedio: 5.0
Cantidad de estrategias rata: 7.0
Cantidad de estrategias antirrata: 7.0
Cantidad de estrategias azarosa: 5.0
Cantidad de estrategias mala_onda: 0.0
Cantidad de estrategias promedio: 6.0
Cantidad de estrategias rata: 6.0
Cantidad de estrategias antirrata: 8.0
Cantidad de estrategias azarosa: 3.0
Cantidad de estrategias mala_onda: 0.0
Cantidad de estrategias promedio: 5.0
Cantidad de estrategias rata: 8.0
Cantidad de estrategias antirrata: 11.0
Cantidad de estrategias azarosa: 2.0
Cantidad de estrategias mala_onda: 0.0
Cantidad de estrategias promedio: 4.0
Cantidad de estrategias rata: 8.0
Cantidad de estrategias antirrata: 11.0
Cantidad de estrategias azarosa: 2.0
Cantidad de estrategias mala_onda: 0

{'azarosa': (<function estrategias.estrat_proponer_rand(agente, vecino)>,
  <function estrategias.estrat_aceptar_rand(agente, vecino, valor)>,
  1.0),
 'mala_onda': (<function estrategias.estrat_proponer_rand(agente, vecino)>,
  <function estrategias.estrat_aceptar_nunca(agente, vecino, valor)>,
  0.0),
 'promedio': (<function estrategias.estrat_proponer_promedio(agente, vecino)>,
  <function estrategias.estrat_aceptar_promedio(agente, vecino, valor)>,
  1.0),
 'rata': (<function estrategias.estrat_rata(agente, vecino)>,
  <function estrategias.estrat_aceptar_promedio(agente, vecino, valor)>,
  0.0),
 'antirrata': (<function estrategias.estrat_antirrata_proponer(agente, vecino)>,
  <function estrategias.estrat_antirrata_aceptar(agente, vecino, valor)>,
  23.0)}